# Illustration of the contains function in MOCpy

## Install libraries locally because they are not yet on micropip

In [ ]:
import micropip
from pathlib import Path

In [ ]:
whl_paths = list(Path.cwd().glob(pattern="wheels/*"))
whl_paths

In [ ]:
await micropip.install("emfs:" + str(path) for path in whl_paths)

In [ ]:
from astropy.coordinates import SkyCoord, Latitude, Longitude
import astropy.units as u
import numpy as np
from mocpy import MOC
import matplotlib.pyplot as plt
from astropy.wcs.utils import skycoord_to_pixel

In [ ]:
def generate_rand_points(num_points: int, lon_min=-10, lon_max=10, lat_min=-10, lat_max=10):
    """Generate a random number of points in a square region of the sky.

    Parameters
    ----------
    num_points : int
        The number of points to be generated
    lon_min : int, optional
        Minimum longitude, by default -10
    lon_max : int, optional
        Maximum longitude, by default 10
    lat_min : int, optional
        Minimum lattitude, by default -10
    lat_max : int, optional
        Maximum lattitude, by default 10

    Returns
    -------
    (astropy.units.quantity.Quantity, astropy.units.quantity.Quantity)
        Tuple of longitudes and lattitudes
    """
    lon = (np.random.random(num_points) * (lon_max - lon_min) + lon_min) * u.deg
    lat = (np.random.random(num_points) * (lat_max - lat_min) + lat_min) * u.deg
    return lon, lat

In [ ]:
lon = Longitude([5, -5, -5, 5], u.deg)
lat = Latitude([5, 5, -5, -5], u.deg)
polygon_moc = MOC.from_polygon(lon, lat)

In [ ]:
# Generate the points
lon, lat = generate_rand_points(10000)
# Convert to Skycoord object for later use in plot
coordinates = SkyCoord(ra=lon, dec=lat, frame='icrs')
# Calculation of contains_lonlat to the random points
mask = polygon_moc.contains_lonlat(lon, lat)
# Green for True, red for False, and apply that to the mask (might not be the most efficient way to do this)
colors = {True: 'g', False: 'r'}
mask_colors = [colors[bool_contains] for bool_contains in mask]

In [ ]:
fig = plt.figure(figsize=(10, 10))
wcs = polygon_moc.wcs(fig)
x, y = skycoord_to_pixel(coordinates, wcs) # to plot the points on same wcs
ax = fig.add_subplot(projection=wcs)
ax.grid(True)
polygon_moc.fill(ax, wcs, fill=True, color='blue', alpha=0.5)
ax.scatter(x, y, c=mask_colors, alpha=0.5)
polygon_moc.border(ax, wcs, color='blue')